# Convex Regression: Learning Convex Functions from Data

This notebook demonstrates **convex regression** — learning a convex function that best fits scattered data, using CVXPYlayers to enforce convexity constraints.

**Model**: We represent the convex function as a max-of-affines:

$$f_\theta(x) = \max_j \; (a_j x + b_j)$$

where $\theta = \{a_j, b_j\}_{j=1}^K$ are learnable PyTorch parameters. Since the pointwise max of affine functions is always convex, this model is *convex by construction*.

**Approach**: We evaluate $f_\theta$ directly in PyTorch (using `torch.max`), and use a CvxpyLayer to solve a **convex projection** problem — projecting noisy data onto the epigraph of $f_\theta$ to get denoised predictions.

**Training**: Learn $\theta$ via SGD to minimize prediction error.

In [ ]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from cvxpylayers.torch import CvxpyLayer

## Generate Data from a Convex Function

We sample data from a known convex function $g(x) = x^2/4 + |x|$ with added noise.

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

n_data = 150

X_np = np.random.uniform(-3, 3, size=(n_data, 1))

def true_convex(x):
    return 0.25 * x[:, 0] ** 2 + np.abs(x[:, 0])

Y_np = true_convex(X_np) + 0.3 * np.random.randn(n_data)

X_data = torch.tensor(X_np, dtype=torch.float64)
Y_data = torch.tensor(Y_np, dtype=torch.float64)

print(f"{n_data} data points generated.")

## Max-of-Affines Evaluation (PyTorch)

The function $f_\theta(x) = \max_j (a_j x + b_j)$ is differentiable almost everywhere and can be computed directly in PyTorch. The slopes $a_j$ and intercepts $b_j$ are learnable parameters.

In [ ]:
K = 8  # number of affine pieces

# Learnable parameters
a_tch = nn.Parameter(torch.randn(K, 1, dtype=torch.float64) * 0.5)
b_tch = nn.Parameter(torch.randn(K, dtype=torch.float64) * 0.5)

def max_of_affines(x, a, b):
    """Evaluate f(x) = max_j (a_j^T x + b_j) for a batch of x.
    
    Args:
        x: (batch, input_dim)
        a: (K, input_dim) slopes
        b: (K,) intercepts
    Returns:
        (batch,) function values
    """
    # (batch, K) = (batch, input_dim) @ (input_dim, K) + (K,)
    vals = x @ a.T + b.unsqueeze(0)
    return vals.max(dim=1).values

print(f"Max-of-{K}-affines model with {2*K} parameters.")

## CvxpyLayer: Convex Projection

Given a noisy observation $y$ and a convex function $f_\theta$, we can *project* $y$ onto the epigraph of $f_\theta$ at point $x$ by solving:

$$\min_z \; (z - y)^2 \quad \text{s.t.} \; z \geq c$$

where $c$ is the convex function value $f_\theta(x)$. This is a parametric QP with parameter $c$ (the function value) and $y$ (the target). The CvxpyLayer allows gradients to flow back through this projection.

In [ ]:
z = cp.Variable(name="z")
c_param = cp.Parameter(name="lower_bound")
y_param = cp.Parameter(name="target")

# Project y onto [c, inf) — closest point to y that's >= c
objective = cp.Minimize(cp.square(z - y_param))
constraints = [z >= c_param]
proj_problem = cp.Problem(objective, constraints)
assert proj_problem.is_dpp()

proj_layer = CvxpyLayer(proj_problem, parameters=[c_param, y_param], variables=[z])
print("Projection layer created.")

## Training Loop

We train the max-of-affines parameters to minimize MSE. The CvxpyLayer projection clips predictions to be at least as large as the convex function value, acting as a convexity-aware regularizer.

In [ ]:
optimizer = torch.optim.Adam([a_tch, b_tch], lr=0.05)
losses = []

n_epochs = 300
batch_size = 32

for epoch in range(n_epochs):
    idx = torch.randperm(n_data)[:batch_size]
    X_batch = X_data[idx]
    Y_batch = Y_data[idx]

    optimizer.zero_grad()
    
    # Evaluate max-of-affines
    f_vals = max_of_affines(X_batch, a_tch, b_tch)
    
    # Project targets through CvxpyLayer
    projected = []
    for i in range(batch_size):
        (z_val,) = proj_layer(f_vals[i], Y_batch[i])
        projected.append(z_val)
    projected = torch.stack(projected).squeeze()
    
    loss = torch.mean((projected - Y_batch) ** 2)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if (epoch + 1) % 75 == 0:
        print(f"Epoch {epoch+1:4d} | Loss: {loss.item():.4f}")

## Visualize the Learned Convex Function

In [ ]:
# Evaluate on a dense grid
x_grid = torch.linspace(-3.5, 3.5, 200, dtype=torch.float64).unsqueeze(1)

with torch.no_grad():
    y_learned = max_of_affines(x_grid, a_tch, b_tch).numpy()

y_true = true_convex(x_grid.numpy())

# Individual affine pieces
a_np = a_tch.detach().numpy()
b_np = b_tch.detach().numpy()
x_plot = x_grid.numpy().flatten()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left: fit vs true
ax1.scatter(X_np, Y_np, alpha=0.3, s=15, c='gray', label='Data')
ax1.plot(x_plot, y_true, 'b--', linewidth=2, label='True function')
ax1.plot(x_plot, y_learned, 'r-', linewidth=2, label='Learned (max-of-affines)')
for j in range(K):
    y_affine = a_np[j, 0] * x_plot + b_np[j]
    ax1.plot(x_plot, y_affine, '--', alpha=0.2, color='orange', linewidth=0.8)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Convex Regression Fit')
ax1.legend()
ax1.set_ylim(-1, 7)
ax1.grid(True, alpha=0.3)

# Right: training loss
ax2.plot(losses, 'g-', alpha=0.7)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MSE Loss')
ax2.set_title('Training Loss')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final evaluation on all data
with torch.no_grad():
    f_all = max_of_affines(X_data, a_tch, b_tch)
final_mse = torch.mean((f_all - Y_data) ** 2).item()
print(f"Final MSE on all data: {final_mse:.4f}")
print(f"Learned {K} affine pieces with slopes: {a_np.flatten().round(3)}")